In [26]:
folder_path = "../../data/jss/imrg/ds1/testbed_1_s"
output_file = "results/meal.csv"

In [27]:
import os, csv, time
import numpy as np
from mealpy import PSO, TS, SA, ACOR, GA, Problem, PermutationVar


class JSSP(Problem):
    def __init__(self, bounds, minmax, name="JSSP", processing_times=None, **kwargs):
        self.name = name
        self.processing_times = processing_times
        super().__init__(bounds, minmax, **kwargs)

    def obj_func(self, solution):
        solution = self.decode_solution(solution)["per"]
        num_jobs = len(solution)
        num_machines = len(self.processing_times[0])
        
        # Initialize completion times matrix
        completion_times = [[0] * num_machines for _ in range(num_jobs)]
        total_completion_time = 0

        for job_index in range(num_jobs):
            job = solution[job_index]

            for machine in range(num_machines):
                if job_index == 0:
                    if machine == 0:
                        completion_times[job_index][machine] = self.processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = completion_times[job_index][machine-1] + self.processing_times[job][machine]
                else:
                    if machine == 0:
                        completion_times[job_index][machine] = completion_times[job_index-1][machine] + self.processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = self.processing_times[job][machine] + max(completion_times[job_index][machine-1], completion_times[job_index-1][machine])

                if machine == num_machines - 1:
                    total_completion_time += completion_times[job_index][machine]

        return total_completion_time
    
    def amend_position(self, solution):
        unique_jobs = np.unique(solution)
        if len(unique_jobs) == len(solution):
            # The solution is already a valid permutation
            return solution
        else:
            # If it's not, we reconstruct a valid permutation
            # This scenario might be rare depending on how the solution is generated or modified
            n = len(solution)
            valid_permutation = np.random.permutation(n)
            return valid_permutation.tolist()

In [28]:
def read_file_input(filepath):
    with open(filepath) as f:
        lines = f.readlines()
        sizes = [int(nr) for nr in lines[0].split()]
        m, n = sizes[0], sizes[1]
        arr = np.zeros((n, m))

        for i in range(1, len(lines)):
            arr[i-1, :] = [int(nr) for nr in lines[i].split()]

    return arr

In [29]:
def read_all_files_in_folder(folder_path):
    # List to hold file contents
    file_contents = {}

    # Check if the provided path is a directory
    if not os.path.isdir(folder_path):
        raise ValueError(f"The provided path '{folder_path}' is not a valid directory.")

    # Iterate over all files in the directory
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if it's a file
        if os.path.isfile(file_path):
            try:
                # Open and read the file
                with open(file_path, 'r') as file:
                    content = file.read()
                    file_contents[filename] = content
            except Exception as e:
                print(f"Error reading file {filename}: {e}")
    
    return file_contents

In [37]:
def process_files(file_data):
    results = []
    total_time = 0
    for filename, content in file_data.items():
        data = read_file_input(f"{folder_path}/{filename}")
        problem_jssp = JSSP(bounds=PermutationVar(valid_set=[0, 1, 2, 3, 4], name="per"), processing_times=data, minmax="min", log_to="")
        model = SA.OriginalSA(epoch=1400, temp_init=100, cooling_rate=0.995, step_size=0.1)
        # FIXME: seems like tabu search is bugged
        # model = TS.OriginalTS(epoch=1000, pop_size=50, tabu_size=5, neighbour_size=20, perturbation_scale=0.05)
        # model = GA.BaseGA(epoch=1000, pop_size=50, pc=0.9, pm=0.05, selection="tournament")
        # model = ACOR.OriginalACOR(epoch=1000, pop_size=50, sample_count = 25, intent_factor = 0.5, zeta = 1.0)

        start_time = time.time()
        model.solve(problem=problem_jssp)
        end_time = time.time()
        duration = end_time - start_time

        print(f"Processing file: {filename} took {duration:.4f}s")

        results.append({"filename": filename, "cost": model.g_best.target.fitness, "time": duration})
        total_time += duration

    print(f"Total time spent solving: {total_time:.4f}s")
    return results

In [31]:
def write_results_to_csv(results, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['filename', 'cost', 'time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in results:
            writer.writerow(result)

In [32]:
file_data = read_all_files_in_folder(folder_path)

In [39]:
results = process_files(file_data)

Processing file: t1s_0238.txt took 0.3547s
Processing file: t1s_0216.txt took 0.3038s
Processing file: t1s_0312.txt took 0.2640s
Processing file: t1s_0281.txt took 0.2768s
Processing file: t1s_0113.txt took 0.3662s
Processing file: t1s_0082.txt took 0.2957s
Processing file: t1s_0192.txt took 0.2635s
Processing file: t1s_0266.txt took 0.2448s
Processing file: t1s_0021.txt took 0.2540s
Processing file: t1s_0018.txt took 0.3600s
Processing file: t1s_0162.txt took 0.2362s
Processing file: t1s_0147.txt took 0.2421s
Processing file: t1s_0183.txt took 0.2630s
Processing file: t1s_0346.txt took 0.3537s
Processing file: t1s_0062.txt took 0.3022s
Processing file: t1s_0027.txt took 0.2203s
Processing file: t1s_0200.txt took 0.2511s
Processing file: t1s_0111.txt took 0.2784s
Processing file: t1s_0213.txt took 0.3533s
Processing file: t1s_0034.txt took 0.2339s
Processing file: t1s_0295.txt took 0.2339s
Processing file: t1s_0057.txt took 0.2404s
Processing file: t1s_0328.txt took 0.3285s
Processing 

In [40]:
results.sort(key=lambda x: x["filename"])
write_results_to_csv(results, output_file)

# Results

| temp_init | cooling_rate | epoch | Time |
|-----------|--------------|------------|------|
| 1000 | 0.9995 | 41500 | 12352.20s |
| 1000 | 0.9995 | 27500 | 2280.10s |
| 100 | 0.9995 | 13800 | 1869.28s |
| 100 | 0.999 | 6900 | 587.67s |
| 100 | 0.995 | 1400 | 112.56s |
